In [1]:
from project1_model import ResNet, BasicBlock 
import numpy as np, torch, torchvision 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
trainingdata = torchvision.datasets.CIFAR10('./CIFAR10/', 
                                                 train=True, 
                                                 download=True, 
                                                 transform=torchvision.transforms.ToTensor()) 

testdata = torchvision.datasets.CIFAR10('./CIFAR10/', 
                                             train=False, 
                                             download=True, 
                                             transform=torchvision.transforms.ToTensor()) 


Files already downloaded and verified
Files already downloaded and verified


In [3]:
print(len(trainingdata))
print(len(testdata))

trainDataLoader = torch.utils.data.DataLoader(trainingdata, 
                                              batch_size=64, 
                                              shuffle=True)

testDataLoader = torch.utils.data.DataLoader(testdata, 
                                             batch_size=64, 
                                             shuffle=False) 


50000
10000


In [4]:
images, labels = iter(trainDataLoader).next() 

In [5]:
net = ResNet(BasicBlock, [2, 2, 2, 2]) 
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [6]:
EPOCHS = 2 

In [7]:
train_loss_history = []
test_loss_history = []
accuracy_history = []

for epoch in range(EPOCHS):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(trainDataLoader):
        if i<20: 
            images, labels = data
            images = images 
            labels = labels 
            optimizer.zero_grad()
            predicted_output = net(images)
            fit = Loss(predicted_output,labels)
            fit.backward()
            optimizer.step()
            train_loss += fit.item()
    for i, data in enumerate(testDataLoader):
        if i<20: 
            with torch.no_grad():
                images, labels = data
                images = images 
                labels = labels 
                predicted_output = net(images)
                fit = Loss(predicted_output,labels)
                test_loss += fit.item()
    train_loss = train_loss/len(trainDataLoader)
    test_loss = test_loss/len(testDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    _, indices = torch.max(predicted_output, 1)
    accuracy = torch.tensor(torch.sum(indices == labels).item() / len(predicted_output))
    accuracy_history.append(accuracy)

    print('Epoch %s, Train loss %.3f, Test loss %.3f, Accuracy %.3f'%(epoch, train_loss, test_loss, accuracy))

Epoch 0, Train loss 0.056, Test loss 0.255, Accuracy 0.266
Epoch 1, Train loss 0.051, Test loss 0.236, Accuracy 0.312


In [8]:
plt.plot(range(EPOCHS),train_loss_history,'-',linewidth=3,label='Train error')
plt.plot(range(EPOCHS),test_loss_history,'-',linewidth=3,label='Test error')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid(True)
plt.legend()
plt.savefig('models/run1/loss_plot')
plt.close()

In [9]:
plt.plot(range(EPOCHS),accuracy_history,'-',linewidth=3,label='Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.grid(True)
plt.legend()
plt.savefig('models/run1/accuracy_plot')
plt.close()

In [10]:
torch.save(net.state_dict(), 'models/run1/full_resnet.pt')